```{autolink-concat}
```

::::{margin}
:::{card} Single-channel amplitude model fit with $P$-vector dynamics
TR-030
^^^
Comparison between fit performance for an amplitude model with Breit–Wigner and $P$-vector dynamics. In both cases, data is generated with $P$-vector dynamics.
+++
🚧&nbsp;[compwa.github.io#278](https://github.com/ComPWA/compwa.github.io/pull/278)
:::
::::

# P-vector model fit

In [ ]:
%pip install -q 'qrules[viz]==0.10.2' 'tensorwaves[jax,phsp]==0.4.12' ampform==0.15.4 pandas==2.2.2 sympy==1.12

In [ ]:
from __future__ import annotations

import logging
import os
import re
from collections import defaultdict
from typing import Any

import ampform
import attrs
import graphviz
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import qrules
import sympy as sp
from ampform.dynamics.builder import TwoBodyKinematicVariableSet
from ampform.helicity import ParameterValues
from ampform.io import aslatex, improve_latex_rendering
from ampform.kinematics.phasespace import Kallen
from ampform.sympy import perform_cached_doit, unevaluated
from attrs import define, field
from IPython.display import Markdown, Math, display
from matplotlib import cm
from qrules.particle import Particle
from sympy import Abs
from tensorwaves.data import (
    IntensityDistributionGenerator,
    SympyDataTransformer,
    TFPhaseSpaceGenerator,
    TFUniformRealNumberGenerator,
    TFWeightedPhaseSpaceGenerator,
)
from tensorwaves.estimator import UnbinnedNLL
from tensorwaves.function.sympy import create_parametrized_function
from tensorwaves.interface import DataSample, FitResult, ParametrizedFunction
from tensorwaves.optimizer import Minuit2

logging.getLogger("absl").setLevel(logging.ERROR)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
improve_latex_rendering()

In [ ]:
%config InlineBackend.figure_formats = ['svg']

## Studied decay

In [ ]:
PARTICLE_DB = qrules.load_default_particles()
PARTICLE_DB.update(qrules.io.load("030/additional-definitions.yml"))

In [ ]:
reaction = qrules.generate_transitions(
    initial_state="J/psi(1S)",
    final_state=["eta", "p", "p~"],
    allowed_intermediate_particles=[
        "N**1",
        "N**3",
        "N(1650)+",
        "N(1900)+",
    ],
    allowed_interaction_types=["strong"],
    formalism="helicity",
    particle_db=PARTICLE_DB,
)

In [ ]:
dot = qrules.io.asdot(reaction, collapse_graphs=True)
graphviz.Source(dot)

## Amplitude builder

In [ ]:
@define
class DynamicsSymbolBuilder:
    collected_symbols: set[sp.Symbol, tuple[Particle, TwoBodyKinematicVariableSet]] = (
        field(factory=lambda: defaultdict(set))
    )

    def __call__(
        self, resonance: Particle, variable_pool: TwoBodyKinematicVariableSet
    ) -> tuple[sp.Expr, dict[sp.Symbol, float]]:
        jp = render_jp(resonance)
        charge = resonance.charge
        if variable_pool.angular_momentum is not None:
            L = sp.Rational(variable_pool.angular_momentum)
            X = sp.Symbol(Rf"X_{{{jp}, Q={charge:+d}}}^{{l={L}}}")
        else:
            X = sp.Symbol(Rf"X_{{{jp}, Q={charge:+d}}}")
        self.collected_symbols[X].add((resonance, variable_pool))
        parameter_defaults = {}
        return X, parameter_defaults


def render_jp(particle: Particle) -> str:
    spin = sp.Rational(particle.spin)
    j = (
        str(spin)
        if spin.denominator == 1
        else Rf"\frac{{{spin.numerator}}}{{{spin.denominator}}}"
    )
    if particle.parity is None:
        return f"J={j}"
    p = "-" if particle.parity < 0 else "+"
    return f"J^P={{{j}}}^{{{p}}}"

In [ ]:
model_builder = ampform.get_builder(reaction)
model_builder.adapter.permutate_registered_topologies()
model_builder.config.scalar_initial_state_mass = True
model_builder.config.stable_final_state_ids = [0, 1, 2]
create_dynamics_symbol = DynamicsSymbolBuilder()
for name in reaction.get_intermediate_particles().names:
    model_builder.set_dynamics(name, create_dynamics_symbol)
model = model_builder.formulate()
model.intensity.cleanup()

In [ ]:
selected_amplitudes = {
    k: v for i, (k, v) in enumerate(model.amplitudes.items()) if i == 0
}
Math(aslatex(selected_amplitudes, terms_per_line=1))

In [ ]:
for symbol, resonances in create_dynamics_symbol.collected_symbols.items():
    display(symbol)
    src = "| resonance | mass | width |\n"
    src += "|:---|---:|--:|\n"
    for p, _ in resonances:
        src += f"| ${p.latex}$ | {p.mass:g} GeV | {p.width:g} GeV |\n"
    display(Markdown(src))

## Dynamics parametrization

### Phasespace factor

In [ ]:
@unevaluated(real=False)
class PhaseSpaceCM(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho^\mathrm{{CM}}_{{{m1},{m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return -16 * sp.pi * sp.I * ChewMandelstam(s, m1, m2)


@unevaluated(real=False)
class ChewMandelstam(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\Sigma\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        q = BreakupMomentum(s, m1, m2)
        return (
            1
            / (16 * sp.pi**2)
            * (
                (2 * q / sp.sqrt(s))
                * sp.log(Abs((m1**2 + m2**2 - s + 2 * sp.sqrt(s) * q) / (2 * m1 * m2)))
                - (m1**2 - m2**2) * (1 / s - 1 / (m1 + m2) ** 2) * sp.log(m1 / m2)
            )
        )


@unevaluated(real=False)
class BreakupMomentum(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"q\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt(Kallen(s, m1**2, m2**2)) / (2 * sp.sqrt(s))

In [ ]:
s, m1, m2 = sp.symbols("s m1 m2", nonnegative=True)
exprs = [
    PhaseSpaceCM(s, m1, m2),
    ChewMandelstam(s, m1, m2),
    BreakupMomentum(s, m1, m2),
]
Math(aslatex({e: e.doit(deep=False) for e in exprs}))

### Relativistic Breit-Wigner

In [ ]:
@unevaluated(real=False)
class ChannelWidth(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    width: Any
    _latex_repr_ = R"\Gamma_s\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2, width = self.args
        return width * PhaseSpaceCM(s, m1, m2)


width = sp.Symbol("Gamma0", nonnegative=True)
expr = ChannelWidth(s, m1, m2, width)
Math(aslatex({expr: expr.doit(deep=False)}))

In [ ]:
PARAMETERS_BW = dict(model.parameter_defaults)

In [ ]:
def formulate_breit_wigner(
    resonances: list[tuple[Particle, TwoBodyKinematicVariableSet]],
) -> sp.Expr:
    (_, variables), *_ = resonances
    s = variables.incoming_state_mass**2
    m1 = variables.outgoing_state_mass1
    m2 = variables.outgoing_state_mass2
    m = [sp.Symbol(Rf"m_{{{p.latex}}}") for p, _ in resonances]
    Γ0 = [sp.Symbol(Rf"\Gamma_{{{p.latex}}}") for p, _ in resonances]
    Γ = [ChannelWidth(s, m1, m2, _w) for _w in Γ0]
    β = [sp.Symbol(Rf"\beta_{{{p.latex}}}") for p, _ in resonances]
    expr = sum(
        (β_ * m_ * Γ_) / (m_**2 - s - m_ * Γ0_) for m_, Γ_, Γ0_, β_ in zip(m, Γ0, Γ, β)
    )
    for i, (resonance, _) in enumerate(resonances):
        PARAMETERS_BW[β[i]] = 1 + 0j
        PARAMETERS_BW[m[i]] = resonance.mass
        PARAMETERS_BW[Γ0[i]] = resonance.width
    return expr

In [ ]:
dynamics_expressions_bw = {
    symbol: formulate_breit_wigner(resonances)
    for symbol, resonances in create_dynamics_symbol.collected_symbols.items()
}
model_bw = attrs.evolve(
    model,
    parameter_defaults=ParameterValues({
        **model.parameter_defaults,
        **PARAMETERS_BW,
    }),
)
Math(aslatex(dynamics_expressions_bw))

In [ ]:
full_expression_bw = perform_cached_doit(model_bw.expression).xreplace(
    dynamics_expressions_bw
)
sp.count_ops(full_expression_bw)

### $P$ vector

In [ ]:
PARAMETERS_F = dict(model.parameter_defaults)

In [ ]:
def formulate_k_matrix(
    resonances: list[tuple[Particle, TwoBodyKinematicVariableSet]],
) -> sp.Expr:
    (_, variables), *_ = resonances
    s = variables.incoming_state_mass**2
    m = [sp.Symbol(Rf"m_{{{p.latex}}}") for p, _ in resonances]
    g = [sp.Symbol(Rf"g_{{{p.latex}}}") for p, _ in resonances]

    expr = sum((g_**2) / (m_**2 - s) for m_, g_ in zip(m, g))
    for i, (resonance, _) in enumerate(resonances):
        PARAMETERS_F[m[i]] = resonance.mass
        PARAMETERS_F[g[i]] = 1
    return expr

In [ ]:
def formulate_p_vector(
    resonances: list[tuple[Particle, TwoBodyKinematicVariableSet]],
) -> sp.Expr:
    (_, variables), *_ = resonances
    s = variables.incoming_state_mass**2
    g = [sp.Symbol(Rf"g_{{{p.latex}}}") for p, _ in resonances]
    m = [sp.Symbol(Rf"m_{{{p.latex}}}") for p, _ in resonances]
    β = [sp.Symbol(Rf"\beta_{{{p.latex}}}") for p, _ in resonances]
    expr = sum((g_ * β_) / (m_**2 - s) for m_, g_, β_ in zip(m, g, β))
    for i, (resonance, _) in enumerate(resonances):
        PARAMETERS_F[β[i]] = 1 + 0j
        PARAMETERS_F[m[i]] = resonance.mass
        PARAMETERS_F[g[i]] = 1
    return expr

In [ ]:
def formulate_f_vector(
    resonances: list[tuple[Particle, TwoBodyKinematicVariableSet]],
) -> sp.Expr:
    (_, variables), *_ = resonances
    s = variables.incoming_state_mass**2
    m1 = variables.outgoing_state_mass1
    m2 = variables.outgoing_state_mass2
    rho = PhaseSpaceCM(s, m1, m2)
    K = formulate_k_matrix(resonances)
    P = formulate_p_vector(resonances)
    return (1 / (1 - rho * K)) * P

In [ ]:
dynamics_expressions_fvector = {
    symbol: formulate_f_vector(resonances)
    for symbol, resonances in create_dynamics_symbol.collected_symbols.items()
}
model_fvector = attrs.evolve(
    model,
    parameter_defaults=ParameterValues({
        **model.parameter_defaults,
        **PARAMETERS_F,
    }),
)
Math(aslatex(dynamics_expressions_fvector))

In [ ]:
full_expression_fvector = perform_cached_doit(model_fvector.expression).xreplace(
    dynamics_expressions_fvector
)
sp.count_ops(full_expression_fvector)

### Create numerical functions

In [ ]:
intensity_expr_bw = perform_cached_doit(full_expression_bw)
intensity_func_bw = create_parametrized_function(
    expression=intensity_expr_bw,
    backend="jax",
    parameters=PARAMETERS_BW,
)

In [ ]:
intensity_expr_fvector = perform_cached_doit(full_expression_fvector)
intensity_func_fvector = create_parametrized_function(
    expression=intensity_expr_fvector,
    backend="jax",
    parameters=PARAMETERS_F,
)

##  Generate data

### Generate phase space sample

In [ ]:
rng = TFUniformRealNumberGenerator(seed=0)
phsp_generator = TFPhaseSpaceGenerator(
    initial_state_mass=reaction.initial_state[-1].mass,
    final_state_masses={i: p.mass for i, p in reaction.final_state.items()},
)
phsp_momenta = phsp_generator.generate(100_000, rng)

epsilon = 1e-8
transformer = SympyDataTransformer.from_sympy(model.kinematic_variables, backend="jax")
phsp = transformer(phsp_momenta)
phsp = {k: v + epsilon * 1j if re.match(r"^m_\d\d$", k) else v for k, v in phsp.items()}

### Update function parameters

In [ ]:
new_parameters_bw = {
    R"\Gamma_{N(1650)^{+}}": 1 / 1.65,
    R"\Gamma_{N(1900)^+}": 1 / 1.9,
    R"\Gamma_{N^{**}_1}": 1 / 1.85,
    R"\Gamma_{N^{**}_3}": 1 / 1.75,
    R"m_{N(1650)^{+}}": 1.65,
    R"m_{N(1900)^+}": 1.9,
    R"m_{N^{**}_1}": 1.85,
    R"m_{N^{**}_3}": 1.75,
}
intensity_func_bw.update_parameters(new_parameters_bw)

In [ ]:
new_parameters_fvector = {
    R"\beta_{N(1650)^{+}}": 1 + 0j,
    R"\beta_{N(1900)^+}": 1 + 0j,
    R"\beta_{N^{**}_1}": 1 + 0j,
    R"\beta_{N^{**}_3}": 1 + 0j,
    R"g_{N(1650)^{+}}": 1.65,
    R"g_{N(1900)^+}": 1,
    R"g_{N^{**}_1}": 1,
    R"g_{N^{**}_3}": 1,
    R"m_{N(1650)^{+}}": 1.65,
    R"m_{N(1900)^+}": 1.9,
    R"m_{N^{**}_1}": 1.95,
    R"m_{N^{**}_3}": 1.75,
}
intensity_func_fvector.update_parameters(new_parameters_fvector)

## Plot sub-intensities

In [ ]:
def compute_sub_intensity(
    func: ParametrizedFunction,
    input_data: DataSample,
    resonances: list[str],
    coupling_pattern: str,
):
    original_parameters = dict(func.parameters)
    negative_lookahead = f"(?!{'|'.join(map(re.escape, resonances))})"
    # https://regex101.com/r/WrgGyD/1
    pattern = rf"^{coupling_pattern}({negative_lookahead}.)*$"
    set_parameters_to_zero(func, pattern)
    array = func(input_data)
    func.update_parameters(original_parameters)
    return array


def set_parameters_to_zero(func: ParametrizedFunction, name_pattern: str) -> None:
    new_parameters = dict(func.parameters)
    for par_name in func.parameters:
        if re.match(name_pattern, par_name) is not None:
            new_parameters[par_name] = 0
    func.update_parameters(new_parameters)

In [ ]:
total_intensities_bw = intensity_func_bw(phsp)
sub_intensities_bw = {
    p: compute_sub_intensity(
        intensity_func_bw,
        phsp,
        resonances=[p.latex],
        coupling_pattern=r"\\beta",
    )
    for symbol, resonances in create_dynamics_symbol.collected_symbols.items()
    for p, _ in resonances
}

In [ ]:
total_intensities_fvector = intensity_func_fvector(phsp)
sub_intensities_fvector = {
    p: compute_sub_intensity(
        intensity_func_fvector,
        phsp,
        resonances=[p.latex],
        coupling_pattern=r"\\beta",
    )
    for symbol, resonances in create_dynamics_symbol.collected_symbols.items()
    for p, _ in resonances
}

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.set_xlim(2, 5)
ax.set_xlabel(R"$m_{p\eta}^{2}$ [GeV$^2$]")
ax.set_ylabel(R"Intensity [a. u.]")
ax.set_yticks([])

bins = 150
phsp_projection = np.real(phsp["m_01"]) ** 2
ax.hist(
    phsp_projection,
    weights=total_intensities_fvector,
    alpha=0.2,
    bins=bins,
    color="hotpink",
    label="Full intensity $F$ vector",
)
ax.hist(
    phsp_projection,
    weights=total_intensities_bw,
    alpha=0.2,
    bins=bins,
    color="grey",
    label="Full intensity Breit-Wigner",
)
ax.hist(
    len(sub_intensities_fvector) * [phsp_projection],
    weights=list(sub_intensities_fvector.values()),
    alpha=0.6,
    bins=bins,
    histtype="step",
    label=[
        Rf"Resonance at ${p.mass}\,\mathrm{{GeV}}$ $F$ vector"
        for p in sub_intensities_fvector
    ],
)

ax.hist(
    len(sub_intensities_bw) * [phsp_projection],
    weights=list(sub_intensities_bw.values()),
    alpha=0.6,
    bins=bins,
    histtype="step",
    label=[
        Rf"Resonance at ${p.mass}\,\mathrm{{GeV^2}}$ Breit-Wigner"
        for p in sub_intensities_fvector
    ],
    ls="dotted",
)

fig.legend(loc="upper right")
plt.tight_layout()
plt.show()

### Dynamics expressions

In [ ]:
dynamics_expr_bw, *_ = dynamics_expressions_bw.values()
dynamics_expr_bw

In [ ]:
dynamics_expr_fvector, *_ = dynamics_expressions_fvector.values()
dynamics_expr_fvector

In [ ]:
dynamics_func_bw = create_parametrized_function(
    expression=dynamics_expr_bw.doit(),
    backend="jax",
    parameters=model_bw.parameter_defaults,
    use_cse=False,
)

In [ ]:
dynamics_func_fvector = create_parametrized_function(
    expression=dynamics_expr_fvector.doit(),
    backend="jax",
    parameters=model_fvector.parameter_defaults,
    use_cse=False,
)

### Weighted data with $F$ vector 

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
ax.hist(
    phsp["m_01"].real,
    bins=100,
    weights=np.real(intensity_func_fvector(phsp)),
)
ax.set_xlabel(R"$M^2\left(\eta p\right)\, \mathrm{[(GeV/c)^2]}$")
ax.set_ylabel(R"Intensity [a.u.]")
fig.tight_layout()
fig.show()

In [ ]:
weighted_phsp_generator = TFWeightedPhaseSpaceGenerator(
    initial_state_mass=model.reaction_info.initial_state[-1].mass,
    final_state_masses={i: p.mass for i, p in model.reaction_info.final_state.items()},
)
data_generator = IntensityDistributionGenerator(
    domain_generator=weighted_phsp_generator,
    function=intensity_func_fvector,
    domain_transformer=transformer,
)
data_momenta = data_generator.generate(50_000, rng)
data = transformer(data_momenta)

In [ ]:
resonances = sorted(
    model.reaction_info.get_intermediate_particles(),
    key=lambda p: p.mass,
)
evenly_spaced_interval = np.linspace(
    0, 1, len(intensity_func_fvector.parameters.items())
)
colors = [cm.rainbow(x) for x in evenly_spaced_interval]
fig, ax = plt.subplots(figsize=(9, 4))
ax.hist(
    np.real(data["m_01"]),
    bins=200,
    alpha=0.5,
    density=True,
)
ax.set_xlabel("$m$ [GeV]")
for (k, v), color in zip(new_parameters_bw.items(), colors):
    if k.startswith("m_{"):
        ax.axvline(
            x=v,
            linestyle="dotted",
            label=r"$" + k + "$",
            color=color,
        )
ax.legend()
plt.show()

## Perform fit

### Estimator definition

In [ ]:
estimator_bw = UnbinnedNLL(
    intensity_func_bw,
    data=data,
    phsp=phsp,
    backend="jax",
)
estimator_fvector = UnbinnedNLL(
    intensity_func_fvector,
    data=data,
    phsp=phsp,
    backend="jax",
)

In [ ]:
reaction_info = model.reaction_info
resonances = sorted(
    reaction_info.get_intermediate_particles(),
    key=lambda p: p.mass,
)
evenly_spaced_interval_F = np.linspace(
    0, 1, len(intensity_func_fvector.parameters.items())
)
colors_F = [cm.rainbow(x) for x in evenly_spaced_interval_F]
evenly_spaced_interval_BW = np.linspace(0, 1, len(intensity_func_bw.parameters.items()))
colors_BW = [cm.gist_rainbow(x) for x in evenly_spaced_interval_BW]


def indicate_masses(ax):
    ax.set_xlabel("$m$ [GeV]")
    for (k, v), color_F in zip(intensity_func_fvector.parameters.items(), colors_F):
        if k.startswith("m_{"):
            ax.axvline(
                x=v,
                linestyle="dotted",
                label=r"$" + k + "$" "(F vector)",
                color=color_F,
            )
    for (k, v), color_BW in zip(intensity_func_bw.parameters.items(), colors_BW):
        if k.startswith("m_{"):
            ax.axvline(
                x=v,
                linestyle="dotted",
                label=r"$" + k + "$" "(Breit-Wigner)",
                color=color_BW,
            )


def compare_model(
    variable_name,
    data,
    phsp,
    function1,
    function2,
    bins=100,
):
    intensities1 = function1(phsp)
    intensities2 = function2(phsp)
    _, ax = plt.subplots(figsize=(9, 4))
    data_projection = np.real(data[variable_name])
    ax = plt.gca()
    ax.hist(
        data_projection,
        bins=bins,
        alpha=0.5,
        label="data",
        density=True,
    )
    phsp_projection = np.real(phsp[variable_name])
    ax.hist(
        phsp_projection,
        weights=np.array(intensities1),
        bins=bins,
        histtype="step",
        color="red",
        label="Fit model with K matrix",
        density=True,
    )
    ax.hist(
        phsp_projection,
        weights=np.array(intensities2),
        bins=bins,
        histtype="step",
        color="blue",
        label="Fit model with Breit Wigner",
        density=True,
    )
    indicate_masses(ax)
    ax.legend()

### Initial parameters

In [ ]:
initial_parameters_bw = {
    R"m_{N^{**}_1}": 1.8,
    R"\Gamma_{N^{**}_1}": 1 / 1.85,
    R"m_{N(1900)^+}": 1.93,
    R"\Gamma_{N(1900)^+}": 1 / 1.93,
    R"m_{N^{**}_3}": 1.7,
    R"\Gamma_{N^{**}_3}": 1 / 1.65,
    R"m_{N(1650)^{+}}": 1.6,
    R"\Gamma_{N(1650)^{+}}": 1 / 1.6,
}
initial_parameters_fvector = {
    R"m_{N^{**}_1}": 1.95,
    R"m_{N^{**}_3}": 1.7,
    R"m_{N(1650)^{+}}": 1.67,
    R"m_{N(1900)^+}": 1.91,
    R"\beta_{N^{**}_3}": 1 + 0j,
    R"\beta_{N(1650)^{+}}": 1 + 0j,
    R"\beta_{N(1900)^+}": 1 + 0j,
    R"g_{N^{**}_1}": 1.0,
    R"g_{N^{**}_3}": 1,
    R"g_{N(1650)^{+}}": 1.6,
    R"g_{N(1900)^+}": 1.0,
}

In [ ]:
original_parameters_bw = dict(intensity_func_bw.parameters)
intensity_func_bw.update_parameters(initial_parameters_bw)
original_parameters_fvector = dict(intensity_func_fvector.parameters)
intensity_func_fvector.update_parameters(initial_parameters_fvector)
compare_model("m_01", data, phsp, intensity_func_fvector, intensity_func_bw)

### Optimize parameters

In [ ]:
minuit2 = Minuit2()

In [ ]:
fit_result_bw = minuit2.optimize(estimator_bw, initial_parameters_bw)
fit_result_bw

In [ ]:
fit_result_fvector = minuit2.optimize(estimator_fvector, initial_parameters_fvector)
fit_result_fvector

In [ ]:
intensity_func_fvector.update_parameters(fit_result_fvector.parameter_values)
intensity_func_bw.update_parameters(fit_result_bw.parameter_values)
compare_model("m_01", data, phsp, intensity_func_fvector, intensity_func_bw)

### Fit result comparison

In [ ]:
def compute_aic_bic(fit_result: FitResult) -> tuple[float, float]:
    n_real_par = fit_result.count_number_of_parameters(complex_twice=True)
    n_events = len(next(iter(data.values())))
    log_likelihood = -fit_result.estimator_value
    aic = 2 * n_real_par - 2 * log_likelihood
    bic = n_real_par * np.log(n_events) - 2 * log_likelihood
    return aic, bic

In [ ]:
def compare_parameters(original: dict, initial: dict, optimized: dict) -> pd.DataFrame:
    parameters = sorted(set(initial) | set(optimized))
    df = pd.DataFrame(
        {
            f"${p}$": (
                initial.get(p, "NaN"),
                optimized.get(p, "NaN"),
                original.get(p, "NaN"),
            )
            for p in parameters
        },
    ).T
    df.columns = ("initial", "fit result", "original")
    return df

In [ ]:
compute_aic_bic(fit_result_fvector)

In [ ]:
compare_parameters(
    original=original_parameters_fvector,
    initial=initial_parameters_fvector,
    optimized=fit_result_fvector.parameter_values,
)

In [ ]:
compute_aic_bic(fit_result_bw)

In [ ]:
compare_parameters(
    original=original_parameters_bw,
    initial=initial_parameters_bw,
    optimized=fit_result_bw.parameter_values,
)